In [1]:
import pandas as pd
import numpy as np
import re
import gensim
from gensim.parsing.preprocessing import remove_stopwords

In [2]:
data = pd.read_excel("D:\Data science\Projects docs\Project chatbot\Chatbot_data.xlsx")

In [3]:
data.head()

,Questions,Answers
0,1. What Is A Recommender System,A recommender system is today widely deployed...
1,"2. Compare Sas, R and Python Programming",SAS: it is one of the most widely used analyt...
2,3. Explain the Various Benefits of R Language,The R programming language includes a set of ...
3,4. How Do Data Scientists Use Statistics,Statistics helps Data Scientists to look into...
4,5. What Is Logistic Regression,It is a statistical technique or a model in o...


In [4]:
data.shape

(92, 2)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92 entries, 0 to 91
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Questions  92 non-null     object
 1   Answers    92 non-null     object
dtypes: object(2)
memory usage: 1.6+ KB


In [6]:
# Cleaning the sentences

In [7]:
def clean_sentences(sentence, stop_words = False):
    sentence = sentence.lower().strip()
    sentence = re.sub('[^a-z0-9\s]', '', sentence)
    
    if stop_words:
        sentence = remove_stopwords(sentence)
    
    return sentence

In [8]:
def get_cleaned_sentences(data, stop_words = False):
    sents = data[["Questions"]]
    cleaned_sentences = []
    
    for index, row in data.iterrows():
        cleaned = clean_sentences(row["Questions"], stop_words)
        cleaned = re.sub("[0-9]", "", cleaned)
        cleaned_sentences.append(cleaned)
    return cleaned_sentences

In [9]:
# cleaned questions without stop words
cleaned_sentences = get_cleaned_sentences(data, stop_words = True)

In [10]:
cleaned_sentences[0:10]

[' recommender',
 ' compare sas r python programming',
 ' explain benefits r language',
 ' data scientists use statistics',
 ' logistic regression',
 ' data cleansing important data analysis',
 ' univariate bivariate multivariate analysis',
 ' machine learning deployed real world scenarios',
 ' aspects machine learning process',
 ' understand term normal distribution']

In [11]:
# cleaned questions with stopwords
cleaned_sentenes_with_stopwords = get_cleaned_sentences(data, stop_words = False)

In [12]:
cleaned_sentenes_with_stopwords[0:10]

[' what is a recommender system',
 ' compare sas r and python programming',
 ' explain the various benefits of r language',
 ' how do data scientists use statistics',
 ' what is logistic regression',
 ' why data cleansing is important in data analysis',
 ' describe univariate bivariate and multivariate analysis',
 ' how machine learning is deployed in real world scenarios',
 ' what are the various aspects of a machine learning process',
 ' what do you understand by the term normal distribution']

In [13]:
sentences = cleaned_sentenes_with_stopwords

In [14]:
# Split the words by white space

In [15]:
sentence_words = [[word for word in document.split()] for document in sentences]

In [16]:
# Creating dictionary of words

In [17]:
from gensim import corpora

In [18]:
dictionary = corpora.Dictionary(sentence_words)

In [19]:
for key, value in dictionary.items():
    print(key, ":", value)

0 : a
1 : is
2 : recommender
3 : system
4 : what
5 : and
6 : compare
7 : programming
8 : python
9 : r
10 : sas
11 : benefits
12 : explain
13 : language
14 : of
15 : the
16 : various
17 : data
18 : do
19 : how
20 : scientists
21 : statistics
22 : use
23 : logistic
24 : regression
25 : analysis
26 : cleansing
27 : important
28 : in
29 : why
30 : bivariate
31 : describe
32 : multivariate
33 : univariate
34 : deployed
35 : learning
36 : machine
37 : real
38 : scenarios
39 : world
40 : are
41 : aspects
42 : process
43 : by
44 : distribution
45 : normal
46 : term
47 : understand
48 : you
49 : linear
50 : extrapolation
51 : interpolation
52 : power
53 : can
54 : for
55 : k
56 : kmeans
57 : select
58 : database
59 : design
60 : different
61 : from
62 : modeling
63 : feature
64 : vectors
65 : decision
66 : making
67 : steps
68 : tree
69 : cause
70 : root
71 : crossvalidation
72 : collaborative
73 : filtering
74 : all
75 : at
76 : converge
77 : descent
78 : gradient
79 : methods
80 : point
81 : 

In [20]:
bow_corpus = [dictionary.doc2bow(text) for text in sentence_words]

In [21]:
for sent, embedding in zip(sentences, bow_corpus):
    print(sent)
    print(embedding)
    print()

 what is a recommender system
[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)]

 compare sas r and python programming
[(5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1)]

 explain the various benefits of r language
[(9, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1)]

 how do data scientists use statistics
[(17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1)]

 what is logistic regression
[(1, 1), (4, 1), (23, 1), (24, 1)]

 why data cleansing is important in data analysis
[(1, 1), (17, 2), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1)]

 describe univariate bivariate and multivariate analysis
[(5, 1), (25, 1), (30, 1), (31, 1), (32, 1), (33, 1)]

 how machine learning is deployed in real world scenarios
[(1, 1), (19, 1), (28, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1)]

 what are the various aspects of a machine learning process
[(0, 1), (4, 1), (14, 1), (15, 1), (16, 1), (35, 1), (36, 1), (40, 1), (41, 1), (42, 1)]

 what do you understand by the term normal distribution

## Bag of words embedding

In [22]:
import sklearn
from sklearn.metrics.pairwise import cosine_similarity

In [23]:
# Defining a function to get the answer

In [24]:
pd.options.display.max_colwidth = None

def PrintAnswer(question_embedding, sentence_embedding, df, sentences):
    max_sim = 1
    ind_sim = -1 
    for index, faq_embedding in enumerate(sentence_embedding):
        sim = cosine_similarity(faq_embedding, question_embedding)[0][0]
        print(index, sim, sentences[index])
        if sim > max_sim:
            max_sim = sim
            ind_sim = index
    print('\n')
    print('Question: ', question)
    print('\n')
    print('Retrieved ', df.iloc[ind_sim, 0])
    print(df.iloc[ind_sim, 1])            

In [25]:
# Testing the function with a question

In [26]:
question = "why data cleansing is important in data analysis"
question = clean_sentences(question, stop_words = True)
question_embedding = dictionary.doc2bow(question.split())

In [27]:
print(question, '\n', question_embedding)

data cleansing important data analysis 
 [(17, 2), (25, 1), (26, 1), (27, 1)]


In [28]:
PrintAnswer(question_embedding, bow_corpus, data, sentences)

0 0.1168412475673972  what is a recommender system
1 0.9967787445474133  compare sas r and python programming
2 0.9999791889951382  explain the various benefits of r language
3 0.9982979513032774  how do data scientists use statistics
4 0.7848827655334262  what is logistic regression
5 0.7848827655334262  why data cleansing is important in data analysis
6 0.9967787445474133  describe univariate bivariate and multivariate analysis
7 0.7848827655334262  how machine learning is deployed in real world scenarios
8 0.1168412475673972  what are the various aspects of a machine learning process
9 0.991835775307426  what do you understand by the term normal distribution
10 0.7848827655334262  what is linear regression
11 0.7848827655334262  what is interpolation and extrapolation
12 0.7848827655334262  what is power analysis
13 0.7848827655334262  what is kmeans  how can you select k for kmeans
14 0.7848827655334262  how is data modeling different from database design
15 0.991835775307426  what

#### "BOW" method is not working properly, hence another method can be used

## 'Glove' and 'Word2Vec' embedding

In [29]:
from gensim.models import word2vec
import gensim.downloader as api

In [30]:
# Downloading Glove and Word2Vec models

In [31]:
glove_model = None
try:
    glove_model = gensim.model.KeyedVectors.load('./glovemodel.mod')
    print("Loaded Glove Model")
except:
    glove_model = api.load("glove-twitter-25")
    glove_model.save("./glovemodel.mod")
    print("Saved Glove Model")

Saved Glove Model


In [32]:
v2w_model = None
try:
    v2w_model = gensim.model.KeyedVectors.load('./w2vecmodel.mod')
    print("Loaded v2w Model")
except:
    v2w_model = api.load("word2vec-google-news-300")
    v2w_model.save('./w2vecmodel.mod')
    print("Saved v2w Model")

[==================================================] 100.0% 1662.8/1662.8MB downloaded
Saved v2w Model


In [33]:
w2vec_embedding_size = len(v2w_model['computer'])
glove_embedding_size = len(glove_model['computer'])

In [34]:
# Defining function to take question and retrieve answer

In [35]:
def getWordvec(word, model):
    samp = model['computer']
    vec = [0]*len(samp)
    try:
        vec = model[word]
    except:
        vec = [0]*len(samp)
    return (vec)

In [36]:
def getPhraseEmbedding(phrase, embeddingmodel):
    samp = getWordvec('computer', embeddingmodel)
    vec = np.array([0]*len(samp))
    den = 0
    for word in phrase.split():
        den = den+1
        vec = vec+np.array(getWordvec(word, embeddingmodel))
    return vec.reshape(1, -1)

In [ ]:
# Retrieving relevant question and answer with Word2Vec model

In [37]:
sent_embeddings = []
for sent in cleaned_sentences:
    sent_embeddings.append(getPhraseEmbedding(sent, v2w_model))

question_embedding = getPhraseEmbedding(question, v2w_model)

In [40]:
PrintAnswer(question_embedding, sent_embeddings, data, cleaned_sentences)

0 0.2767839148134592  recommender
1 0.18974432860954218  compare sas r python programming
2 0.2709074773592258  explain benefits r language
3 0.6974599068160648  data scientists use statistics
4 0.31533645833365354  logistic regression
5 1.0000000000000002  data cleansing important data analysis
6 0.5232633654234448  univariate bivariate multivariate analysis
7 0.30585343340666665  machine learning deployed real world scenarios
8 0.33050077664146615  aspects machine learning process
9 0.31022701003870323  understand term normal distribution
10 0.2961850807011226  linear regression
11 0.2915158876889972  interpolation extrapolation
12 0.5848069589504791  power analysis
13 0.09370918701021415  kmeans select k kmeans
14 0.6759878173315197  data modeling different database design
15 0.22180762994886202  feature vectors
16 0.19314132010618892  explain steps making decision tree
17 0.47140846268992737  root cause analysis
18 0.1082753264234256  explain crossvalidation
19 0.3686834580123191  

In [41]:
# Retrieving relevant question and answer with Glove model

In [42]:
sent_embeddings = []
for sent in cleaned_sentences:
    sent_embeddings.append(getPhraseEmbedding(sent, glove_model))

question_embedding = getPhraseEmbedding(question, glove_model)

In [43]:
PrintAnswer(question_embedding, sent_embeddings, data, cleaned_sentences)

0 0.0  recommender
1 0.7906008816119312  compare sas r python programming
2 0.8051940294381376  explain benefits r language
3 0.9047266490771675  data scientists use statistics
4 0.5503523856669136  logistic regression
5 1.0000000000000004  data cleansing important data analysis
6 0.8762196779298578  univariate bivariate multivariate analysis
7 0.7728242751003606  machine learning deployed real world scenarios
8 0.9050118345161404  aspects machine learning process
9 0.851060469415186  understand term normal distribution
10 0.5955369823620535  linear regression
11 0.0  interpolation extrapolation
12 0.927302544897734  power analysis
13 0.6184479288055786  kmeans select k kmeans
14 0.9096092613148183  data modeling different database design
15 0.6481355473096122  feature vectors
16 0.7731342956127127  explain steps making decision tree
17 0.8291412880145512  root cause analysis
18 0.6273381969826294  explain crossvalidation
19 0.7053778512714018  collaborative filtering
20 0.820619950598